<a href="https://colab.research.google.com/github/SamEspe/big-data-challenge/blob/main/Big_Data_Challenge_Toys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data Challenge: Amazon Toy Reviews

### Set Up Google Colab Environment

In [1]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (83.3 kB/s)
Reading package li

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 02:49:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-30 02:49:17 (10.5 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

## Import Amazon Toy Reviews

In [4]:
from pyspark import SparkFiles
 
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

toys_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep = "\t", header = True, inferSchema = True)

toys_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# Count the number of rows

rows = toys_reviews_df.count()
rows

4864249

## Create Dataframes for SQL Tables

In [6]:
# Import PySpark SQL functionality to add count column and change column data types
from pyspark.sql.functions import col, lit
from pyspark.sql.types import DateType

In [7]:
# Add customer count column
toys_reviews_df = toys_reviews_df.withColumn("customer_count", lit(rows))

In [8]:
# Create review_id table
review_id_table = toys_reviews_df["review_id", "customer_id", "product_id", "product_parent", "review_date"]

In [9]:
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]

In [10]:
# Change review_date column to date type

review_id_table = review_id_table.withColumn("review_date", col("review_date").cast(DateType()))

In [11]:
# Create products table

products_table = toys_reviews_df["product_id", "product_title"]

In [32]:
products_table.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [13]:
# Create customers table

customers_table = toys_reviews_df["customer_id", "customer_count"]

In [33]:
customers_table.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [15]:
# Create vine table
vine_table = toys_reviews_df["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]

In [17]:
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

## PostgreSQL Set Up

In [25]:
# Save connection parameters

aws_endpoint = "amazon-toy-reviews.ca5i2jfd0gkp.us-east-2.rds.amazonaws.com"
aws_port_number = 5432
aws_database_name = "amazon_toy_reviews"
aws_username = 'postgres'
aws_password = 'bootcamp'

In [26]:
# Make the connection string

jdbc_url = f'jdbc:postgresql://{aws_endpoint}:{aws_port_number}/{aws_database_name}'

config = {
    "user": f'{aws_username}',
    "password": f'{aws_password}',
    "driver": "org.postgresql.Driver"
}

mode = "overwrite"

In [28]:
review_id_table.write.jdbc(url = jdbc_url, table = "review_id_table", mode = mode, properties = config)

In [29]:
products_table.write.jdbc(url = jdbc_url, table = 'products', mode = mode, properties = config)

In [30]:
customers_table.write.jdbc(url = jdbc_url, table = 'customers', mode = mode, properties = config)

In [31]:
vine_table.write.jdbc(url = jdbc_url, table = 'vine_table', mode = mode, properties = config)